In [30]:
import numpy as np

class huiseguanlianfenxi():
    def __init__(self, sequence_cankao, sequence_bijiao, xi=0.5):
        """
        初始化灰色关联分析模型。
        
        参数:
        sequence_cankao (list or np.array): 参考序列（母序列）。
        sequence_bijiao: 比较序列（子序列）。子序列的每一行：为影响因素/指标，列为每一个方案对应的指标值
        xi (float, optional): 分辨系数，取值范围一般为0到1，常用值为0.5。

        母序列为因变量（数据）关于自变量（方案集）的变化趋势

        示例：
                     |2016, 2017, 2018, 2019|  #方案集
    国民生产总值 =    |55,    65,   75,   100|  #母序列
        工业产值 =    |24,    38,   40,   50 |  #子序列
        农业产值 =    |10,    22,   18,   20 |  #子序列
        """

        #把输入的母序列和子序列都转化为array
        #其中，母序列为1*n数组，n为方案集长度
        #子序列为x*n数组，x为指标数，n为方案集长度
        self.sequence_cankao = np.array(sequence_cankao)
        self.sequence_bijiao = np.array([np.array(seq) for seq in sequence_bijiao])
        self.xi = xi  # 分辨系数
        self.sequence_cankao_guiyihua=None  #归一化的参考矩阵，在"#归一化"模块得到
        self.sequence_bijiao_guiyihua=None  #归一化的比较矩阵，在"#归一化"模块得到
        self.matrix_chazhi=None #差值矩阵，在"#计算差值矩阵"模块得到
        self.matrix_huise=None #灰色矩阵，在"#计算灰色关联系数"模块得到
        self.guanliandu=None   #关联度，有几个比较指标就有几个关联度，示例中关联度为2。在"#计算关联度与权重"模块得到
        self.weights=None      #权重，关联度归一化得到。在"#计算关联度与权重"模块得到

        
        
        # 检查序列长度是否一致
        self.jianchachangdu_function()

    #检查序列长度是否匹配
    def jianchachangdu_function(self):
        """检查所有序列的长度是否一致."""
        len_cankaoxulie = len(self.sequence_cankao)
        
        #遍历矩阵中每一个子序列(每一行)
        for seq in self.sequence_bijiao:
            if len(seq) != len_cankaoxulie:
                raise ValueError("所有序列的长度必须一致。")

    #归一化
    def normalize_function(self):
         #对母序列归一化   
        sequence_cankao1=self.sequence_cankao/np.mean(self.sequence_cankao)

        #归一化矩阵传回self
        self.sequence_cankao_guiyihua=sequence_cankao1
        
        #对子序列做归一化
        matrix=[]
        for index in range(len(self.sequence_bijiao)):#这行代码用于遍历各个指标（子序列的行）
            sequence_bijiao1=self.sequence_bijiao[index,:]/np.mean(self.sequence_bijiao[index,:])
            matrix.append(sequence_bijiao1)  #此时的matrix是一个列表

        #将列表转化为数组
        matrix=np.array(matrix)

        #归一化子序列传回self
        self.sequence_bijiao_guiyihua=matrix


    #计算差值矩阵
    def chazhijuzhen_calculate_function(self):
        """计算差值矩阵 matrix_chazhi."""
        matrix_chazhi = []
        for sequence_hang in self.sequence_bijiao_guiyihua:  #seq表示子序列（比较矩阵的每一行）
            
            chazhi = np.abs(self.sequence_cankao_guiyihua - sequence_hang)  #差值的计算公式：母序列-子序列每一行
            matrix_chazhi.append(chazhi)

        #将差值矩阵转化为数组类型
        self.matrix_chazhi = np.array(matrix_chazhi)



    #计算灰色关联系数
    def coefficient_huiseguanlian_function(self):
        '''
        变量解释：
        zuixiaocha：两极最小差
        zuidacha：两极最大差
        '''
        # 找到差值矩阵中的最小值和最大值
        zuixiaocha = np.min(self.matrix_chazhi)
        zuidacha = np.max(self.matrix_chazhi)

        #构造灰色关联函数并得出矩阵
        matrix_huise=(zuixiaocha+self.xi*zuidacha)/(self.matrix_chazhi+self.xi*zuidacha)

        #将灰色矩阵传入self，使得别的函数可以直接调用
        self.matrix_huise=matrix_huise
        
    #计算关联度与权重
    def guanliandu_function(self):
        
        guanliandu1=[]#初始化空列表
        
        for index in range(len(self.matrix_huise)):  #遍历灰色矩阵每一行
            
            guanliandu1.append(np.mean(self.matrix_huise[index,:]))
        #将关联度传回self
        self.guanliandu=guanliandu1

        #计算权重
        self.weights=guanliandu1/np.sum(guanliandu1)


    # 主函数，执行所有步骤
    def execute(self):
        # 1. 归一化
        self.normalize_function()

        # 2. 计算差值矩阵
        self.chazhijuzhen_calculate_function()

        # 3. 计算灰色关联系数矩阵
        self.coefficient_huiseguanlian_function()

        # 4. 计算关联度与权重
        self.guanliandu_function()

In [32]:
# 定义母序列和比较序列
sequence_cankao = [55, 65, 75, 100]  # 国民生产总值
sequence_bijiao = [
    [24, 38, 40, 50],  # 工业产值
    [10, 22, 18, 20]   # 农业产值
]

# 创建模型实例并执行分析
model = huiseguanlianfenxi(sequence_cankao, sequence_bijiao)
#执行灰色关联分析的所有方法
model.execute()  

# 输出结果
print("关联度:", model.guanliandu)
print("权重:", model.weights)

关联度: [0.7696657818674999, 0.6005846425336497]
权重: [0.56169717 0.43830283]
